In [2]:
%pip install pygraphviz

import pygraphviz as pgv



Note: you may need to restart the kernel to use updated packages.


In [16]:
log = "/home/wdecoster/temp/slurm-2201.out"

trees = {}
seq_weights = {}

with open(log) as f:
    for line in f:
        if "DEBUG STRdust::phase_insertions] " in line and "with dissimilarity" in line:
            graph = True
            name = line.split(' ')[3].rstrip(':')
            if name not in trees:
                trees[name] = pgv.AGraph(directed=True)
                trees[name].node_attr['style']='filled'
                seq_weights[name] = {}
            node = line.split(' ')[5]
            children = (line.split(' ')[11], line.split(' ')[14])
            seqs = (line.split(' ')[12], line.split(' ')[15].rstrip('\n'))
            trees[name].add_edge(node, children[0])
            trees[name].add_edge(node, children[1])
            for child, seq in zip(children, seqs):
                if len(seq) > 2:
                    seq_weights[name][child] = len(seq)
        if "Roots for this tree:" in line:
            name = line.split(' ')[3].rstrip(':')
            for root in line.split(':')[-1].replace(' ', '').replace('\n', '').replace('[', '').replace(']', '').split(','):
                n = trees[name].get_node(root)
                n.attr['fillcolor']="#CCCCFF"
    for label, G in trees.items():
        maxlength = max(seq_weights[label].values())
        for node, length in seq_weights[label].items():
            color = int((length / maxlength) * 255)
            n = G.get_node(node)
            n.attr['fillcolor']="#%02x%02x%02x" % (255, 255 - color, 255 - color)
        G.layout(prog='dot')
        G.draw(f"graph_{label}.png")
        


In [10]:
import os
os.getcwd()

'/home/wdecoster/wsl-repos/STRdust/misc'